## Imports

In [1]:
# !conda activate n2v
%load_ext autoreload

import numpy as np
# from matplotlib import pyplot as plt
import sys
import random
import zarr
from PIL import Image
from skimage import data
from skimage import filters
from skimage import metrics

import gunpowder as gp

# from this repo
# from segway.tasks.make_zarr_from_tiff import task_make_zarr_from_tiff_volume as tif2zarr
from boilerPlate import GaussBlur

2021-08-11 19:49:43.178530: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# ADD HELPER FUNCTIONS

def noise_pipe(src, target, pipeline, noise_order, noise_dict):
    this_array = src
    for noise in noise_order:
        if noise_dict[noise]:
            if noise =='downX' and noise_dict[noise]:
                pipeline += gp.DownSample(src, (1, noise_dict[noise], noise_dict[noise]), target) # assumes zyx coordinates (and non-isometric)
                this_array = target
            elif noise =='gaussBlur' and noise_dict[noise]:
                pipeline += GaussBlur(this_array, noise_dict[noise])
            elif noise =='gaussNoise' and noise_dict[noise]:
                pipeline += gp.NoiseAugment(this_array, mode='gaussian', var=noise_dict[noise])
            elif noise =='poissNoise' and noise_dict[noise]:
                pipeline += gp.NoiseAugment(this_array, mode='poisson')
            # elif noise =='deform' and noise_dict[noise]: # TODO: IMPLEMENT
            #     pipeline += ...

    return pipeline

# Specify Parameters (source, noise type, downsampling, etc.)

In [3]:
noise_version = '' # for making multiple independently generated noise versions (e.g. for Fourier Shell analysis)
src_path = '/n/groups/htem/ESRF_id16a/tomo_ML/synapse/cb2/' # PATH FOR ZARR

raw_name = 'raw'
noise_dict = {'downX': 8, # cudegy mimic of 30nm pixel size (max uttained) from sensor at ESRF.i16a X-ray source, assuming 4nm voxel size EM source images
         'gaussBlur': 30, # cudegy mimic of 30nm resolution of KB mirrors at ESRF.i16a X-ray source
         'gaussNoise': None, # ASSUMES MEAN = 0, THIS SETS VARIANCE
         'poissNoise': True, # cudegy mimic of sensor shot noise (hot pixels) at ESRF.i16a X-ray source
        #  'deform': , # TODO: IMPLEMENT
         }

# noise_order = ['gaussBlur', 
#                'downX', 
#                'gaussNoise', 
#                'poissNoise'
#                ]

noise_order = ['downX', 
               'gaussBlur', 
               'gaussNoise', 
               'poissNoise'
               ]

samples = [
    'ml0',
    'ml1',
    'cutout1',
    'cutout2',
    'cutout5',
    'cutout6',
    'cutout7',
    ]

src_voxel_size = (40, 4, 4)

In [4]:
##### BELOW IS AUTOMATIC PARAMETER SETUP BASED ON ABOVE SPECIFICATIONS ######

noise_name = ''
for noise in noise_order:
    if noise_dict[noise]:
        if str(noise_dict[noise]).isnumeric():
            noise_name += noise + str(noise_dict[noise]) + '_'
        elif isinstance(noise_dict[noise], bool):
            noise_name += noise + '_'

if noise_version == '':
    noise_name = noise_name[:-1]
else:
    noise_name += noise_version

print('Layer name for noised data: ' + noise_name)

if noise_dict['downX']:
    dest_voxel_size = [src_voxel_size[s] * noise_dict['downX'] if s > 0 else src_voxel_size[s] for s in range(len(src_voxel_size))]
else:
    dest_voxel_size = src_voxel_size
src_voxel_size = gp.Coordinate(src_voxel_size)
dest_voxel_size = gp.Coordinate(dest_voxel_size)


Layer name for noised data: downX8_gaussBlur30_poissNoise


# Setup Noising Pipeline

In [65]:
%autoreload
from boilerPlate import GaussBlur

In [5]:
# declare arrays to use in the pipeline
raw = gp.ArrayKey('RAW') # raw data
noisy = gp.ArrayKey('NOISY') # data noise added
raw_spec = gp.ArraySpec(voxel_size=src_voxel_size, interpolatable=True)

stack = gp.Stack(1)

# request matching the model input and output sizes
scan_request = gp.BatchRequest()
scan_request.add(raw, (40, 256, 256))#, src_voxel_size)
scan_request.add(noisy, (40, 256, 256))#, dest_voxel_size)
# scan_request[noisy].dtype = np.float64

scan = gp.Scan(scan_request)

# request an empty batch from scan
request = gp.BatchRequest()

# assemble pipeline for each volume and run
for sample in samples:    
    # $src_path$volume/$volume.zarr/volumes/$layer
    src = f'{src_path}{sample}/{sample}.zarr/volumes'
    source = gp.ZarrSource(    # add the data source
            src,  # the zarr container
            {raw: raw_name},  # which dataset to associate to the array key
            {raw: raw_spec}  # meta-information
    )

    destination = gp.ZarrWrite(
            dataset_names = {noisy: noise_name},
            output_dir = f'{src_path}{sample}',
            output_filename = f'{sample}.zarr',
            dataset_dtypes = {noisy: np.uint8} # save as 0-255 values (should match raw)
    )

    pipeline = (noise_pipe(raw, noisy, source, noise_order, noise_dict) + 
                # stack + 
                destination + 
                scan)
    with gp.build(pipeline):
        pipeline.request_batch(request)

410it [00:20, 20.61it/s]

In [32]:
down = gp.DownSample(raw, (1,8,8), noisy)
down.prepare(scan_request)
down.spec